# Лабораторная работа 4

#### Оценка тональности рецензий

## Часть 1. Скраппинг данных

Будем доставать отзывы с Кинопоиска. Для анализа были выбраны фильмы, у которых достаточно и хороших, и плохих отзывов: "Ла-ла-ленд" (2016), "Елки" (2010), "Барби" (2023), "Бесславные ублюдки" (2009) и "Оно" (2017). Запиешм их ID в список

In [1]:
ids = [841081, 493768, 478052, 9691, 453397]

Для парсинга будем использовать библиотеку BeautifulSoup4

In [2]:
import requests
from bs4 import BeautifulSoup

Напишем функцию для записи данных в файл.

In [22]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from time import sleep
import re
import os

In [70]:
def load_user_info(movie_ids: list[int], filename: str):
    #checking if files do not exist or are empty 
    does_exist = (os.path.isfile(filename)) and (not os.stat(filename).st_size == 0)
    if does_exist:
        print(f"File {filename} already exists.")
        return

    #creating the list of rows for our future dataframe
    rows = []
    
    for id in tqdm(movie_ids):
        for target in ['good', 'bad']:
            header = {
                "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
                'referer':'https://www.google.com/'
            }
            data = requests.get(f"https://www.kinopoisk.ru/film/{id}/reviews/ord/login/status/{target}/perpage/200/", headers=header)
            soup = BeautifulSoup(data.text)
            title_tag = soup.findAll("title")[0].text
            title = re.sub(" — отзывы и рецензии — Кинопоиск", "", title_tag)

            for tag in soup.findAll("span", {"itemprop" : "reviewBody"}):
                review = tag.text
                rows.append((title, review, target))

            sleep(2)
        
    #creating the dataframe and dumping it to a file
    df = pd.DataFrame.from_dict(rows, orient='columns')
    df.to_csv(filename, index=False)

    return

In [71]:
load_user_info(ids, "reviews.csv")

100%|██████████| 5/5 [00:32<00:00,  6.52s/it]

910


Выгрузим данные обратно в датфрейм

In [76]:
df = pd.read_csv("reviews.csv", names = ['title', 'review', 'status'], skiprows=1)
df.sample(20)

,title,review,status
189,Ла-Ла Ленд (2016),"Художники, поэты, артисты часто кажутся глупца...",good
834,Оно (2017),Роман Стивена Кинга “Оно” акцентировал внимани...,good
599,Бесславные ублюдки (2009),"Совершенно плоский и неинтересный фильм, лживы...",bad
600,Бесславные ублюдки (2009),Для меня существует два Тарантино. Один - это ...,bad
235,Ла-Ла Ленд (2016),"Ох, граждане…. У меня дичайшее deja vu, отсыла...",bad
275,Ёлки (2010),"Пошел в кинотеатр с девушкой. Несмотря на то, ...",good
416,Ёлки (2010),"Тем, у кого одно только упоминание фамилии Бек...",good
398,Ёлки (2010),Изобилие русских знаменитостей в списке актёро...,good
681,Бесславные ублюдки (2009),Об американском тщеславии мы все слышали не од...,bad
642,Бесславные ублюдки (2009),"Честно говоря, ждал этот фильм. Такого разочар...",bad


## Часть 2. Обучение модели